In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
import sys
from pathlib import Path

project_root = os.getenv('PROJECT_ROOT')
sys.path.append(str(project_root))
print(project_root)
print(os.getcwd())

/Users/ivang/PycharmProjects/pythonProject/
/Users/ivang/PycharmProjects/pythonProject/notebooks


# Power analysis for required sample size for Likert scale

In [3]:
import statsmodels.stats.power as smp

# Define the effect size as the correlation coefficient (r)
effect_size = 0.5  # for example, a medium effect size

# Significance level (alpha)
alpha = 0.1

# Desired power
power = 0.8

# Calculate the required sample size
analysis = smp.NormalIndPower()
sample_size = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, alternative='two-sided')

print(f"Required sample size: {sample_size}")

Required sample size: 49.45789459567311


# Load results

In [4]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("Salesforce/SFR-Embedding-Mistral")

/Users/ivang/miniconda3/envs/openml-tags/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/ivang/miniconda3/envs/openml-tags/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
import json

with open('models/topic_model_text_generation_4/topics.json') as f:
    topics_json = json.load(f)

topics_json 

{'topic_representations': {'0': [['Themes: Binarized Regression Problem, Chemistry, Life Science, Mythbusting, Study, Target Feature, Nominal Classification; Overarching themes: Data Analysis, Research, Science, Mathematics',
    1]],
  '1': [['Themes: Anal Cat Data, Binarization, Chemistry, Life Science, Myth Busting, Study; Overarching themes: Data Analysis, Research, Classification',
    1]],
  '2': [['Themes: COVID-19, Pandemic, Cases, Deaths, Data Analysis, Visualization, Vaccinations, Demographics, Economics, Machine Learning, Statistics, Mobility, Tests, Healthcare, Length of Stay, Safety Behavior, No-Show Problem, Education Disruption; Overarching themes: Health, Science, Technology, Society, Policy',
    1]],
  '3': [['Themes: analcatdata, datasets, book, education, research, statistics, manufacturing, automotive accessories, hardware, bolts, counting, electronic eye, sensitivity setting, machine adjustments, time, data formats, comma-delimited, Microsoft Excel, quotation mark

In [6]:
from bertopic import BERTopic

loaded_model = BERTopic.load("models/topic_model_text_generation_4", embedding_model=embedding_model)

In [7]:
import numpy as np

embeddings = np.load('embeddings_prompts.npy')

In [8]:
import pandas as pd

df = pd.read_csv("data/prompt_descriptions.csv")

In [9]:
df

,prompt_description
0,"Anneal \nTags: Manufacturing, Study One, Study..."
1,**Chess End-Game - King + Rook versus King + P...
2,**Labor Negotiations**\n\nThis dataset pertain...
3,"Arrhythmia \n\nTags: Machine Learning, Medicin..."
4,Title: Letter Image Recognition Data\n\nThe ob...
...,...
5278,Credit Score Classification \n\nTags: Person C...
5279,The One Thirty Two dataset has been tagged wit...
5280,Login Data for Cybersecurity Events \n\nTags: ...
5281,Dataset Name: User Ravi Tags: Dummy Data \n\nF...


In [10]:
data = df['prompt_description'].tolist()

In [11]:
topics, probs = loaded_model.transform(data, embeddings)

2024-06-20 19:06:15,817 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


In [12]:
topics

array([808, 782, 160, ..., 138, 851, 837])

In [13]:
probs

array([[0.68619144, 0.6881601 , 0.56816864, ..., 0.5583309 , 0.63750786,
        0.6315758 ],
       [0.61886656, 0.6080779 , 0.5563311 , ..., 0.50684315, 0.56784445,
        0.56537426],
       [0.647854  , 0.65272343, 0.61683375, ..., 0.5501358 , 0.58876204,
        0.59894204],
       ...,
       [0.6562575 , 0.6664578 , 0.62690717, ..., 0.5629943 , 0.61457044,
        0.61581403],
       [0.6390691 , 0.65417254, 0.64584476, ..., 0.5324031 , 0.6081092 ,
        0.6136222 ],
       [0.6957084 , 0.68803996, 0.6052842 , ..., 0.60155207, 0.63383615,
        0.61533713]], dtype=float32)

In [14]:
loaded_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,45,"0_Themes: Binarized Regression Problem, Chemis...","[Themes: Binarized Regression Problem, Chemist...",NaN
1,1,33,"1_Themes: Anal Cat Data, Binarization, Chemist...","[Themes: Anal Cat Data, Binarization, Chemistr...",NaN
2,2,31,"2_Themes: COVID-19, Pandemic, Cases, Deaths, D...","[Themes: COVID-19, Pandemic, Cases, Deaths, Da...",NaN
3,3,23,"3_Themes: analcatdata, datasets, book, educati...","[Themes: analcatdata, datasets, book, educatio...",NaN
4,4,23,"4_Themes: Finance, Forex, Currency Pairs, Hist...","[Themes: Finance, Forex, Currency Pairs, Histo...",NaN
...,...,...,...,...,...
891,891,5,"891_Themes: QSAR, MetaQSAR, Neuropeptide FF Re...","[Themes: QSAR, MetaQSAR, Neuropeptide FF Recep...",NaN
892,892,3,"892_Themes: Music, Geographical Origin, Audio ...","[Themes: Music, Geographical Origin, Audio Fea...",NaN
893,893,4,"893_Themes: Word, Semester, Fifth, Ninth, Seve...","[Themes: Word, Semester, Fifth, Ninth, Seventh...",NaN
894,894,3,"894_Themes: Matrix Metalloproteinase, Stromely...","[Themes: Matrix Metalloproteinase, Stromelysin...",NaN


# TEMPORARY

In [15]:
documents = pd.DataFrame({"Document": data, "Topic": loaded_model.topics_})

In [16]:
repr_docs_mappings, _, _, _ = loaded_model._extract_representative_docs(
            loaded_model.c_tf_idf_,
            documents,
            loaded_model.topic_representations_,
            nr_repr_docs=30
        )

In [17]:
topic = 255
docs = repr_docs_mappings[topic]

In [18]:
labels = sorted(list(documents.Topic.unique()))
labels = [int(label) for label in labels]
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
_, words = loaded_model._c_tf_idf(documents_per_topic)
top_n_words = max(loaded_model.top_n_words, 30)
indices = loaded_model._top_n_idx_sparse(loaded_model.c_tf_idf_, top_n_words)
scores = loaded_model._top_n_values_sparse(loaded_model.c_tf_idf_, indices)
sorted_indices = np.argsort(scores, 1)
indices = np.take_along_axis(indices, sorted_indices, axis=1)
scores = np.take_along_axis(scores, sorted_indices, axis=1)

# Get top 30 words per topic based on c-TF-IDF score
topics_ctfidf = {label: [(words[word_index], score)
                    if word_index is not None and score > 0
                    else ("", 0.00001)
                    for word_index, score in zip(indices[index][::-1], scores[index][::-1])
                    ]
            for index, label in enumerate(labels)}
topics_ctfidf

{0: [('binarized', 0.024385889087135373),
  ('nominal', 0.012931456567301818),
  ('myth', 0.012387009404784557),
  ('busting', 0.012387009404784557),
  ('lower', 0.010693910789245701),
  ('chemistry', 0.010679035312484443),
  ('converted', 0.009983685689582047),
  ('life', 0.009908596275495885),
  ('classifying', 0.008652771983661996),
  ('mythbusting', 0.008642041025063003),
  ('computing', 0.00840450275148269),
  ('science', 0.008286920823062477),
  ('bean', 0.008079681796129102),
  ('achieved', 0.007899530076776637),
  ('version', 0.007865542669744555),
  ('conversion', 0.007838070706105047),
  ('numeric', 0.007779208506827736),
  ('others', 0.007537740104336586),
  ('regression', 0.007507528909051453),
  ('feature', 0.007467474927967779),
  ('negative', 0.0073986865372829965),
  ('theta', 0.0073628982288420075),
  ('problem', 0.007104322834062427),
  ('all', 0.0067741276548872915),
  ('circumference', 0.006670467415510201),
  ('cortex', 0.006626646465094454),
  ('instances', 0.0061

In [37]:
prompt = '''Based on the below information, extract and synthesize human-readable tags/keywords/themes from the text, capitalized first letters of words. What is the main human-readable theme or subject matter discussed in the provided texts? What is the overarching, high-level theme of the texts, e.g. "Music", "Sports", "Environment", etc.? Please provide overarching themes that tie the different pieces of information together. What is/are the overarching, highest level theme(s) that you could use as a keyword(s)? Prefer single word tags/keywords, e.g. "Tennis" rather than "Tennis Match", "Prison" rather than "Prison Time", etc.! Some examples of human-readable themes are   "Agriculture", "Astronomy", "Chemistry", "Computational Universe", "Computer Systems", "Climate and Environment", "Culture", "Demographics", "Earth Science", "Economics", "Education", "Engineering", "Finance", "Geography", "Government", "Games", "Health", "History", "Human Activities", "Images", "Language", "Law", "Life Science", "Machine Learning", "Manufacturing", "Mathematics", "Medicine", "Meteorology", "Physical Sciences", "Politics", "Social Media", "Sociology", "Statistics", "Text & Literature",  "Transportation". Also, don't give very similar tags/keywords, e.g. "Wine" and "Red Wine", just give one or the other in these cases. Avoid tags/keywords that are too specific, e.g. "Serine Threonine Protein Kinase". Good theme examples are: "Birds", "Species Migration", "Air Pollution", or "War", "Government", "International Relations", "Politics". Another important rule to obey - place more focus on the dataset names for theme extraction. And be concise in theme generation, e.g. instead of "Income Prediction", say "Income", instead of "Demographic Information", say "Demographics"! Also, extract the theme of the text, what it is about, instead of the type of problem it is, for instance we don't care about "Regression", "Numerical Features", "Data Analysis", "Data", "Outliers" or things of that sort, but we care about the ESSENCE of the text! Say {"Themes": [...], "Overarching themes": [...]} and give your answer in JSON format.
For example, for this text:
Text 1: The Biden administration is preparing to roll out a sweeping border executive action as early as Tuesday, according to two sources familiar with the discussions, who cautioned that timing is fluid.

White House officials have begun reaching out to mayors who represent cities along the US southern border to potentially join President Joe Biden when he announces the order, two other sources familiar with those conversations said.

For weeks, administration officials have been working through an executive action that would dramatically limit migrants’ ability to seek asylum at the US southern border — part of a strategy to try to give Biden the upper hand on one of his Republican rival’s key campaign issues. The action is designed to potentially blunt Republican attacks on border security and preempt former President Donald Trump ahead of the first presidential debate, which will be held on June 27 on CNN.
---
Text 2: Now that a New York jury has convicted former President Donald Trump of all 34 felony charges of falsifying business records, the next obvious question is: Can a convicted felon run for president?

Definitely.

Trump meets all three requirements. There is, arguably, another criterion laid out in the 14th Amendment, where it states that no one who has previously taken an oath of office who engages in insurrection can be an officer of the US. But the US Supreme Court ruled earlier this year that Congress would have to pass a special law invoking this prohibition. That’s not happening any time soon.

Judge Juan Merchan has scheduled Trump’s sentencing for July 11, which happens to be four days before the start of the Republican National Convention that is scheduled to take place in Milwaukee.
a
It is technically possible, although perhaps unlikely for a first-time offender, that Trump could be sentenced to prison time.
---
This would be your answer:
{"Themes": ["Biden Administration", "Border", "Executive Action", "Asylum", "Immigration", "Trump", "Felony", "Business Records", "Presidential Campaign", "Republican", "Debate", "Former President", "Conviction", "Sentencing", "Prison", "14th Amendment", "Insurrection", "Supreme Court", "Republican National Convention"], "Overarching themes": ["Politics", "Government", "Law", "Justice", "Elections"]}
---
Now, the above was just an example. Now, do it for the following text(s), be concise!:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]
---
Remember, extract and synthesize human-readable tags/keywords/themes from the text, capitalized first letters of words. What is the main human-readable theme or subject matter discussed in the provided texts? What is the overarching, high-level theme of the texts, e.g. "Music", "Sports", "Environment", etc.? Please provide overarching themes that tie the different pieces of information together. What is/are the overarching, highest level theme(s) that you could use as a keyword(s)? Prefer single word tags/keywords, e.g. "Tennis" rather than "Tennis Match", "Prison" rather than "Prison Time", etc.! Some examples of human-readable themes are   "Agriculture", "Astronomy", "Chemistry", "Computational Universe", "Computer Systems", "Climate and Environment", "Culture", "Demographics", "Earth Science", "Economics", "Education", "Engineering", "Finance", "Geography", "Government", "Games", "Health", "History", "Human Activities", "Images", "Language", "Law", "Life Science", "Machine Learning", "Manufacturing", "Mathematics", "Medicine", "Meteorology", "Physical Sciences", "Politics", "Social Media", "Sociology", "Statistics", "Text & Literature",  "Transportation". Also, don't give very similar tags/keywords, e.g. "Wine" and "Red Wine", just give one or the other in these cases. Avoid tags/keywords that are too specific, e.g. "Serine Threonine Protein Kinase". Good theme examples are: "Birds", "Species Migration", "Air Pollution", or "War", "Government", "International Relations", "Politics". Another important rule to obey - place more focus on the dataset names for theme extraction. And be concise in theme generation, e.g. instead of "Income Prediction", say "Income", instead of "Demographic Information", say "Demographics"! Also, extract the theme of the text, what it is about, instead of the type of problem it is, for instance we don't care about "Regression", "Numerical Features", "Data Analysis", "Data", "Outliers" or things of that sort, but we care about the ESSENCE of the text! Say {"Themes": [...], "Overarching themes": [...]} and give your answer in JSON format.
'''

In [38]:
def replace_documents(prompt, docs):
    to_replace = ""
    # for doc in docs:
    #     to_replace += f"- {doc}\n---\n"
    # same, but say Text 1: ... (newline) Text 2: ... (newline) etc.
    for i, doc in enumerate(docs):
        to_replace += f"Text {i+1}: {doc}\n---\n"
    prompt = prompt.replace("[DOCUMENTS]", to_replace)
    return prompt

In [39]:
def create_prompt(docs, topic, topics, prompt):
    keywords = list(zip(*topics[topic]))[0]
    if "[KEYWORDS]" in prompt:
        prompt = prompt.replace("[KEYWORDS]", ", ".join(keywords))
    
    if "[DOCUMENTS]" in prompt:
        prompt = replace_documents(prompt, docs)
    
    return prompt

In [40]:
print(create_prompt(docs, topic, topics_ctfidf, prompt))

Based on the below information, extract and synthesize human-readable tags/keywords/themes from the text, capitalized first letters of words. What is the main human-readable theme or subject matter discussed in the provided texts? What is the overarching, high-level theme of the texts, e.g. "Music", "Sports", "Environment", etc.? Please provide overarching themes that tie the different pieces of information together. What is/are the overarching, highest level theme(s) that you could use as a keyword(s)? Prefer single word tags/keywords, e.g. "Tennis" rather than "Tennis Match", "Prison" rather than "Prison Time", etc.! Some examples of human-readable themes are   "Agriculture", "Astronomy", "Chemistry", "Computational Universe", "Computer Systems", "Climate and Environment", "Culture", "Demographics", "Earth Science", "Economics", "Education", "Engineering", "Finance", "Geography", "Government", "Games", "Health", "History", "Human Activities", "Images", "Language", "Law", "Life Scienc

In [23]:
prompt_2 = '''Now, look at the Themes and Overarching themes you gave me. Modify/add/remove the themes that are very similar to each other, ones that are too specific and long (e.g. "Serine Theonine Protein Kinase"), be concise in theme generation (e.g. instead of "Income Prediction" say "Income", "Demographic Information"->"Demographics", "Myth-busting"->"Myth Busting", "Travel Industry" -> "Travel" etc.). Also, remove themes that are related to the technical description of a text, e.g. "Regression", "Numerical Features", "Data Analysis", "Datasets", "Data", "Prediction", "Data Science", etc.. Instead, we care about keeping descriptive, interpretable, coherent themes. Same JSON format.'''

In [24]:
top_k = 5
top_k_topics = []
for i in range(len(data)):
    top_k_indices = np.argsort(probs[i])[::-1][:top_k]
    top_k_indices = [index for index in top_k_indices]
    top_k_indices = [index for index in top_k_indices if index != -1]
    top_k_topics.append([str(topic) for topic in top_k_indices])

top_k_topics[0]

['808', '489', '64', '89', '626']

In [25]:
def split_themes(data):
    themes_dict = {}
    overarching_themes_dict = {}
    
    for key, value in data['topic_labels'].items():
        # Split the value into themes and overarching themes
        # print(key) 
        themes_part, overarching_part = value.split("; Overarching themes: ")
        
        # Further split to remove the leading key part and "Themes: " label
        themes = themes_part.split("_Themes: ")[1]
        
        # Store in dictionaries
        themes_dict[key] = themes.strip()
        overarching_themes_dict[key] = overarching_part.strip()
    
    return themes_dict, overarching_themes_dict

themes_dict, overarching_themes_dict = split_themes(topics_json)

# Print results (for demonstration purposes)
print("Themes Dictionary:")
for k, v in themes_dict.items():
    print(f"{k}: {v}")

print("\nOverarching Themes Dictionary:")
for k, v in overarching_themes_dict.items():
    print(f"{k}: {v}")

Themes Dictionary:
0: Binarized Regression Problem, Chemistry, Life Science, Mythbusting, Study, Target Feature, Nominal Classification
1: Anal Cat Data, Binarization, Chemistry, Life Science, Myth Busting, Study
2: COVID-19, Pandemic, Cases, Deaths, Data Analysis, Visualization, Vaccinations, Demographics, Economics, Machine Learning, Statistics, Mobility, Tests, Healthcare, Length of Stay, Safety Behavior, No-Show Problem, Education Disruption
3: analcatdata, datasets, book, education, research, statistics, manufacturing, automotive accessories, hardware, bolts, counting, electronic eye, sensitivity setting, machine adjustments, time, data formats, comma-delimited, Microsoft Excel, quotation marks, backslashes, blanks, space replacement, acknowledgement, scientific, educational, non-commercial purposes, analytical research, mythbusting, study, hotel, luxury, Europe, customer reviews, scoring, geographical location, sentiment analysis, recommendation engine, clustering, visualization,

# TEMPORARY

In [26]:
loaded_model.get_topic(6)

[['Themes: Aminopeptidase, Peptides, Anticancer, Metallo, Protease, Transporter, Acid, Inactive, Tripeptidyl, Transport, Clan, Furin, Dipeptidyl, Active, Complement, Peptidase, Dipeptidase, Chemotherapy, Annotations, Cancer, AMP, Enzyme, Multiple, Cytotoxicity, Membranolytic, Amidated, CancerPPD, Thermolysin, Chiral; Overarching themes: Biochemistry, Medicine, Cancer Research, Protein Structure and Function, Drug Discovery, Enzymology, Peptide Biology, Chemotherapy, Metalloproteases.',
  1]]

In [27]:
loaded_model.get_document_info(data)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document
0,"Anneal \nTags: Manufacturing, Study One, Study...",489,"489_Themes: Annealing, Steel, Material Science...","[Themes: Annealing, Steel, Material Science, M...",NaN,"Themes: Annealing, Steel, Material Science, Me...",False
1,**Chess End-Game - King + Rook versus King + P...,782,"782_Themes: Chess End-Game, King + Rook vs. Ki...","[Themes: Chess End-Game, King + Rook vs. King ...",NaN,"Themes: Chess End-Game, King + Rook vs. King +...",False
2,**Labor Negotiations**\n\nThis dataset pertain...,160,"160_Themes: Canada, Survey, Data Disclosure, P...","[Themes: Canada, Survey, Data Disclosure, Prob...",NaN,"Themes: Canada, Survey, Data Disclosure, Proba...",False
3,"Arrhythmia \n\nTags: Machine Learning, Medicin...",135,"135_Themes: Heart Disease, Machine Learning, S...","[Themes: Heart Disease, Machine Learning, Stat...",NaN,"Themes: Heart Disease, Machine Learning, Stati...",False
4,Title: Letter Image Recognition Data\n\nThe ob...,624,"624_Themes: Letter Recognition, Image Classifi...","[Themes: Letter Recognition, Image Classificat...",NaN,"Themes: Letter Recognition, Image Classificati...",False
...,...,...,...,...,...,...,...
5278,Credit Score Classification \n\nTags: Person C...,7,"7_Themes: Credit, Loan, Finance, Data Analysis...","[Themes: Credit, Loan, Finance, Data Analysis,...",NaN,"Themes: Credit, Loan, Finance, Data Analysis, ...",False
5279,The One Thirty Two dataset has been tagged wit...,477,"477_Themes: Computer Systems, Data Sets, Machi...","[Themes: Computer Systems, Data Sets, Machine ...",NaN,"Themes: Computer Systems, Data Sets, Machine L...",False
5280,Login Data for Cybersecurity Events \n\nTags: ...,138,"138_Themes: Astronomy, Network Traffic Analysi...","[Themes: Astronomy, Network Traffic Analysis, ...",NaN,"Themes: Astronomy, Network Traffic Analysis, B...",False
5281,Dataset Name: User Ravi Tags: Dummy Data \n\nF...,138,"138_Themes: Astronomy, Network Traffic Analysi...","[Themes: Astronomy, Network Traffic Analysis, ...",NaN,"Themes: Astronomy, Network Traffic Analysis, B...",False


In [28]:
loaded_model.get_document_info(data).loc[0]

Document                   Anneal \nTags: Manufacturing, Study One, Study...
Topic                                                                    489
Name                       489_Themes: Annealing, Steel, Material Science...
Representation             [Themes: Annealing, Steel, Material Science, M...
Representative_Docs                                                      NaN
Top_n_words                Themes: Annealing, Steel, Material Science, Me...
Representative_document                                                False
Name: 0, dtype: object

In [29]:
with open('results.json') as f:
    results = json.load(f)

with open('overarching_results.json') as f:
    overarching_results = json.load(f)

In [30]:
from sklearn.metrics.pairwise import cosine_similarity
# from tqdm import tqdm

# def filter_similar_labels(labels, scores, threshold = 0.95):
#     embeddings = embedding_model.encode(labels)
    
#     cosine_similarities = cosine_similarity(embeddings)
    
#     similar_labels = set()
#     for i in range(len(labels)):
#         for j in range(i + 1, len(labels)):
#             if cosine_similarities[i, j] > threshold:
#                 if scores[i] > scores[j]:
#                     similar_labels.add(labels[j])
#                 else:
#                     similar_labels.add(labels[i])
    
#     return [label for label in labels if label not in similar_labels], [score for label, score in zip(labels, scores) if label not in similar_labels]

# def filter_results(results, score_threshold=0.5):
#     filtered_results = []

#     for result in tqdm(results):
#         filtered = [(label, score) for label, score in zip(result['labels'], result['scores']) if score > score_threshold]

#         if filtered:
#             labels, scores = zip(*filtered)
#             filtered_labels, filtered_scores = filter_similar_labels(list(labels), list(scores))
#         else:
#             filtered_labels, filtered_scores = [], []

#         filtered_results.append({
#             'sequence': result['sequence'],
#             'labels': filtered_labels,
#             'scores': filtered_scores
#         })

#     return filtered_results

In [31]:
# df_results = pd.DataFrame(filter_results(results))
# df_results

In [32]:
# df_overarching_results = pd.DataFrame(filter_results(overarching_results))
# df_overarching_results

In [33]:
# df_results.to_csv('filtered_results.csv', index=False)
# df_overarching_results.to_csv('filtered_overarching_results.csv', index=False)

In [34]:
df_results = pd.read_csv('filtered_results.csv')
df_results

,sequence,labels,scores
0,"Anneal \nTags: Manufacturing, Study One, Study...","['Anneal', 'Steel', 'Data Sets', 'Alloy', 'Met...","[0.9781602621078491, 0.944840133190155, 0.7416..."
1,"Anneal \nTags: Manufacturing, Study One, Study...",['Studies'],[0.6087751984596252]
2,"Anneal \nTags: Manufacturing, Study One, Study...","['Anneal', 'Steel', 'Alloy', 'Metallurgy', 'Ma...","[0.9781602621078491, 0.944840133190155, 0.7214..."
3,**Chess End-Game - King + Rook versus King + P...,"['Chess End-Game', 'Chess', 'King + Rook vs. K...","[0.9902561902999878, 0.954011082649231, 0.9341..."
4,**Labor Negotiations**\n\nThis dataset pertain...,['Canada'],[0.6595309972763062]
...,...,...,...
5280,Credit Score Classification \n\nTags: Person C...,"['Credit', 'Finance', 'Loan']","[0.9906300902366638, 0.664585530757904, 0.5735..."
5281,The One Thirty Two dataset has been tagged wit...,['Ada'],[0.5196735858917236]
5282,Login Data for Cybersecurity Events \n\nTags: ...,"['Login Data for Cybersecurity Event', 'Comput...","[0.9616353511810303, 0.5064505338668823]"
5283,Dataset Name: User Ravi Tags: Dummy Data \n\nF...,[],[]


In [35]:
df_overarching_results = pd.read_csv('filtered_overarching_results.csv')
df_overarching_results

,sequence,labels,scores
0,"Anneal \nTags: Manufacturing, Study One, Study...","['Manufacturing', 'Science', 'Materials', 'Mat...","[0.9330682158470154, 0.7778840661048889, 0.760..."
1,"Anneal \nTags: Manufacturing, Study One, Study...",['Science'],[0.7778840661048889]
2,"Anneal \nTags: Manufacturing, Study One, Study...","['Manufacturing', 'Material', 'Science', 'Mate...","[0.9330682158470154, 0.8098275661468506, 0.777..."
3,**Chess End-Game - King + Rook versus King + P...,"['Chess', 'Research']","[0.954011082649231, 0.6249885559082031]"
4,**Labor Negotiations**\n\nThis dataset pertain...,[],[]
...,...,...,...
5280,Credit Score Classification \n\nTags: Person C...,['Finance'],[0.664585530757904]
5281,The One Thirty Two dataset has been tagged wit...,['Technology'],[0.8180697560310364]
5282,Login Data for Cybersecurity Events \n\nTags: ...,"['Technology', 'Security', 'Computer']","[0.9751905798912048, 0.9700767993927002, 0.659..."
5283,Dataset Name: User Ravi Tags: Dummy Data \n\nF...,"['Technology', 'Computer']","[0.8555715084075928, 0.5300011038780212]"


In [36]:
# find embedding similarity between "computer science" and "computational science"
embedding_model.encode(["computer science", "computational science"])

# similarity
cosine_similarity([embedding_model.encode("computer science")], [embedding_model.encode("computational science")])

array([[0.95230937]], dtype=float32)

In [44]:
cosine_similarity([embedding_model.encode("Financial market")], [embedding_model.encode("Finance")])

array([[0.89755094]], dtype=float32)

In [45]:
cosine_similarity([embedding_model.encode("Used Car")], [embedding_model.encode("Car")])

array([[0.85056174]], dtype=float32)

In [46]:
cosine_similarity([embedding_model.encode("Analcatdata")], [embedding_model.encode("Anal Cat Data")])

array([[0.88096756]], dtype=float32)

: 